In [ ]:
"""
PySpark oferă:
    Un obiectul de tip Spark Context, folosit pentru interacțiunea cu Spark,.
    Un obiect de bază pentru codul SQL și citirea datelor, numit Spark Session.
    Un obiect de lucru cu date, numit Data Frame, având o interfață similară cu cel de Pandas.
    Funcții gata implementate pentru transformări și expresii.
    Abilitatea de a construi propriile funcții de transformare în Python.
"""


In [ ]:
"""
Pregătire mediu de lucru
Stabilirea conexiunii dintre Google drive si Colab notebook
"""
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalarea PySpark

!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://dlcdn.apache.org/spark/
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"

import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
  .builder \
  .getOrCreate()

spark

In [ ]:
"""
Fiind concepute pentru Big Data, un Data Frame nu încarcă datele.
El reține doar locația și tipul lor.
La fiecare  transformare de date, nou Data Frame va fi creat care reține locația și tipul datelor, precum și lanțul de transformări.
Transformările vor fi efectuate doar în momentul scrierii sau afișării datelor de către executori.

Crearea unui Data Frame Spark dintr-o listă de Python.
"""

data = [
    ['Vali', 23, 'Programator', 4, None, ['Sport', 'Boardgames']],
    ['Vlad', 34, 'Instalator', 11, None, ['Alergare']],
    ['Bea', 29, 'Reporter', 7, True, None]
]

data_df = spark.createDataFrame(data)

data_df.show()


In [ ]:
"""
Citirea fișierelor de tip JSON Lines.
"""
path_json = '/content/drive/MyDrive/Colab Notebooks/Data/Data/practice/json'
data_df_json = spark.read.format('json').load(path_json)

#Display rezultate
data_df_json.show()


In [ ]:
"""
Citirea fișierelor de tip Parquet.
"""
path_parq = '/content/drive/MyDrive/Colab Notebooks/Data/Data/practice/parquet'
data_df = spark.read.format('parquet').load(path_parq)

#Display rezultate
data_df.show()


In [ ]:
"""
Schema Datelor

Întrucât Spark SQL a fost conceput pentru data structurate, el încearcă să determine tipurile de coloane, numită schema datelor,
 în mod automat, din sursele de date la citire. Acest proces nu este necesar pentru formate structurate ca Parquet,
  dar necesar pentru obiecte Python sau date semi-structurate, JSON și CSV.

Pentru a dezactiva determinarea automata a tipurilor de date, la citire, se poate furniza schema:
"""
from pyspark.sql import types as T

data_schema = T.StructType([
    T.StructField('nume', T.StringType(), False),
    T.StructField('varsta', T.IntegerType(), False),
    T.StructField('ocupatie', T.StringType(), False),
    T.StructField('vechime',T.IntegerType(),True),
    T.StructField('inactiv', T.BooleanType(), True),
    T.StructField('extra', T.ArrayType(T.StringType()), True)
]);

# Dacă se furnizează la citire schema, Spark va ignora sau pune NULL pe coloanele care nu se potrivesc sau nu există

data_df = spark.read.format('parquet').schema(data_schema).load(path_parq);

data_df.show()

In [ ]:
"""
Fie prin detectare automată, fie prin furnizarea la citire, fiecare Data Frame va avea întotdeauna o structură a datelor foarte bine definită. Pentru fiecare coloană avem numele, tipul de dată și dacă se permit valori de ”NULL”.

Dacă avem de lucrat cu date care nu au o schemă bine definită, fie date nestructurate, fie date semi-structurate cu multe abateri, atunci NU folosim Data Frame-ul din Spark SQL.

Pentru a afișa structură a datelor / schema datelor pentru un Data Frame, folosim:
"""
data_df.printSchema()



In [ ]:
"""
Data Frame – Colectarea Datelor

Alte metode pe care Spark le ofertă, nu numai pentru depănarea datelor, dar și pentru rarele situații când
 am redus datele și vrem să prelucrăm cu alte librării, sunt cele de colectare a datelor în obiecte de Python pe mașina Master.

Atenție! Datele sunt transferate mai întâi de la executori pe Spark Driver la colectare.
Dacă sunt prea multe date, există pericolul ca procesul de Spark Driver să nu mai facă față și să fie terminat de către sistem.

Pentru a colecta datele într-o listă de Python, folosim:

"""

data_list = data_df.collect()
data_list

In [ ]:
"""
Pentru a colecta datele într-un obiect de Data Frame, dar a librăriei Pandas, folosim:

"""

data_pandas_pdf = data_df.toPandas()
data_pandas_pdf

In [ ]:
"""
Scrierea Datelor

"""

# Scrierea fișierelor de tip JSON Lines.
path_output_json = '/content/drive/MyDrive/Colab Notebooks/Data/Data/output/json'
data_df.write.format('json').mode('overwrite').save(path_output_json)



In [ ]:
# Scrierea fișierelor de tip Parquet.
path_output_parq = '/content/drive/MyDrive/Colab Notebooks/Data/Data/output/parquet'
data_df.write.format('parquet').mode('overwrite').save(path_output_parq)
